In [2]:
'''Deep Dreaming in Keras.
By: the keras team; alterations by Alan Jamieson
'''

from __future__ import print_function

from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
import scipy
import argparse

from keras.applications import inception_v3
from keras import backend as K

#set these two arguments - the first is the path to your image, the second is where to store the results
base_image_path = './IMG_1439.jpg'
result_prefix = './myresult3'

# These are the names of the layers
# for which we try to maximize activation,
# as well as their weight in the final loss
# we try to maximize.
# You can tweak these setting to obtain new visual effects.
settings = {
    'features': {
        'mixed2': 1,
        'mixed3': 3,
        'mixed4': 1,
        'mixed5': 3,
    },
}


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = load_img(image_path)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

K.set_learning_phase(0)

# Build the InceptionV3 network with our placeholder.
# The model will be loaded with pre-trained ImageNet weights.
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)
model.summary

dream = model.input
print('Model loaded.')

# Get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Define the loss.
loss = K.variable(0.)
for layer_name in settings['features']:
    # Add the L2 norm of the features of a layer to the loss.
    assert (layer_name in layer_dict.keys(),
            'Layer ' + layer_name + ' not found in model.')
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(x), 'float32'))
    if K.image_data_format() == 'channels_first':
        loss += coeff * K.sum(K.square(x[:, :, 2: -2, 2: -2])) / scaling
    else:
        loss += coeff * K.sum(K.square(x[:, 2: -2, 2: -2, :])) / scaling

# Compute the gradients of the dream wrt the loss.
grads = K.gradients(loss, dream)[0]
# Normalize gradients.
grads /= K.maximum(K.mean(K.abs(grads)), K.epsilon())

# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)


def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values


def resize_img(img, size):
    img = np.copy(img)
    if K.image_data_format() == 'channels_first':
        factors = (1, 1,
                   float(size[0]) / img.shape[2],
                   float(size[1]) / img.shape[3])
    else:
        factors = (1,
                   float(size[0]) / img.shape[1],
                   float(size[1]) / img.shape[2],
                   1)
    return scipy.ndimage.zoom(img, factors, order=1)


def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('..Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x


"""Process:
- Load the original image.
- Define a number of processing scales (i.e. image shapes),
    from smallest to largest.
- Resize the original image to the smallest scale.
- For every scale, starting with the smallest (i.e. current one):
    - Run gradient ascent
    - Upscale image to the next scale
    - Reinject the detail that was lost at upscaling time
- Stop when we are back to the original size.
To obtain the detail lost during upscaling, we simply
take the original image, shrink it down, upscale it,
and compare the result to the (resized) original image.
"""


# Playing with these hyperparameters will also allow you to achieve new effects
step = 0.01  # Gradient ascent step size
num_octave = 4  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 30  # Number of ascent steps per scale
max_loss = 40.

img = preprocess_image(base_image_path)
if K.image_data_format() == 'channels_first':
    original_shape = img.shape[2:]
else:
    original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)

save_img(result_prefix + '.png', deprocess_image(np.copy(img)))

Model loaded.


<ipython-input-2-6d2ecccb1d1f>:75: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (layer_name in layer_dict.keys(),


FileNotFoundError: [Errno 2] No such file or directory: './IMG_1439.jpg'

In [21]:
#Holiday Image Manipulation using Deep Convolutional Network Inception v3
'''Deep Dreaming in Keras.
By: the keras team; alterations by Alan Jamieson
'''

from __future__ import print_function

from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
import scipy
import argparse

from keras.applications import inception_v3
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
# dimensions of our images.

img_width, img_height = 150, 150



train_data_dir = 'E:\Christmas_images'

validation_data_dir = 'E:\Christmas_images'

nb_train_samples = 2000

nb_validation_samples = 800

epochs = 50

batch_size = 1

if K.image_data_format() == 'channels_first':

    input_shape = (3, img_width, img_height)

else:

    input_shape = (img_width, img_height, 3)

#set these two arguments - the first is the path to your image, the second is where to store the results
base_image_path = './Input_pic.jpg'
result_prefix = './myresult'

# These are the names of the layers
# for which we try to maximize activation,
# as well as their weight in the final loss
# we try to maximize.
# You can tweak these setting to obtain new visual effects.
settings = {
    'features': {
        'mixed2': 1,
        'mixed3': 3,
        'mixed4': 1,
        'mixed5': 3,
    },
}


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = load_img(image_path)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

K.set_learning_phase(0)

# Build the InceptionV3 network with our placeholder.
# The model will be loaded with pre-trained ImageNet weights.
model = inception_v3.InceptionV3(weights=None,
                                 include_top=True, input_shape=input_shape)
model.summary()

model.compile(loss='binary_crossentropy',

              optimizer='rmsprop',

              metrics=['accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 74, 74, 32)   96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 74, 74, 32)   0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

__________________________________________________________________________________________________
batch_normalization_141 (BatchN (None, 7, 7, 160)    480         conv2d_141[0][0]                 
__________________________________________________________________________________________________
activation_136 (Activation)     (None, 7, 7, 160)    0           batch_normalization_136[0][0]    
__________________________________________________________________________________________________
activation_141 (Activation)     (None, 7, 7, 160)    0           batch_normalization_141[0][0]    
__________________________________________________________________________________________________
conv2d_137 (Conv2D)             (None, 7, 7, 160)    179200      activation_136[0][0]             
__________________________________________________________________________________________________
conv2d_142 (Conv2D)             (None, 7, 7, 160)    179200      activation_141[0][0]             
__________

In [30]:
train_datagen = ImageDataGenerator(

    rescale=1. / 255,

    shear_range=0.2,

    zoom_range=0.2,

    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(

    train_data_dir,

    target_size=(img_width, img_height),

    batch_size=batch_size)

print(input_shape)

Found 1117 images belonging to 1000 classes.
(150, 150, 3)


In [31]:
#test_datagen = ImageDataGenerator(rescale=1. / 255)

#validation_generator = test_datagen.flow_from_directory(

#    validation_data_dir,

#    target_size=(img_width, img_height),

#    batch_size=batch_size,

#    class_mode='binary')

#for x_batch, y_batch in datagen.flow_from_directory()
model.fit_generator(

    train_generator,

    steps_per_epoch=nb_train_samples // batch_size,

    epochs=epochs) #,

#    validation_data=validation_generator,

#    validation_steps=nb_validation_samples // batch_size)



model.save_weights('first_try.h5')

Epoch 1/50
  23/2000 [..............................] - ETA: 5:23:02 - loss: nan - acc: 0.0434

C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


 825/2000 [===========>..................] - ETA: 31:51 - loss: nan - acc: 0.0012

C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12 bytes but only got 10. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag))


2000/2000 [==============================] - 2974s 1s/step - loss: nan - acc: 4.9950e-04
Epoch 2/50
2000/2000 [==============================] - 2781s 1s/step - loss: nan - acc: 0.0000e+00
Epoch 3/50
2000/2000 [==============================] - 2770s 1s/step - loss: nan - acc: 0.0000e+00
Epoch 4/50
2000/2000 [==============================] - 2779s 1s/step - loss: nan - acc: 0.0000e+00
Epoch 5/50
2000/2000 [==============================] - 2775s 1s/step - loss: nan - acc: 0.0000e+00
Epoch 6/50
 360/2000 [====>.........................] - ETA: 40:11 - loss: nan - acc: 0.0000e+00

KeyboardInterrupt: 

In [34]:
#model.save_weights('first_try.h5')

model2 = inception_v3.InceptionV3(weights=None,
                                 include_top=False)

model2.load_weights('first_try.h5',by_name =True)
dream = model2.input
print('Model loaded.')

# Get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model2.layers])

# Define the loss.
loss = K.variable(0.)
for layer_name in settings['features']:
    # Add the L2 norm of the features of a layer to the loss.
    assert (layer_name in layer_dict.keys(),
            'Layer ' + layer_name + ' not found in model.')
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(x), 'float32'))
    if K.image_data_format() == 'channels_first':
        loss += coeff * K.sum(K.square(x[:, :, 2: -2, 2: -2])) / scaling
    else:
        loss += coeff * K.sum(K.square(x[:, 2: -2, 2: -2, :])) / scaling

# Compute the gradients of the dream wrt the loss.
grads = K.gradients(loss, dream)[0]
# Normalize gradients.
grads /= K.maximum(K.mean(K.abs(grads)), K.epsilon())

# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)


def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values


def resize_img(img, size):
    img = np.copy(img)
    if K.image_data_format() == 'channels_first':
        factors = (1, 1,
                   float(size[0]) / img.shape[2],
                   float(size[1]) / img.shape[3])
    else:
        factors = (1,
                   float(size[0]) / img.shape[1],
                   float(size[1]) / img.shape[2],
                   1)
    return scipy.ndimage.zoom(img, factors, order=1)


def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('..Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x


"""Process:
- Load the original image.
- Define a number of processing scales (i.e. image shapes),
    from smallest to largest.
- Resize the original image to the smallest scale.
- For every scale, starting with the smallest (i.e. current one):
    - Run gradient ascent
    - Upscale image to the next scale
    - Reinject the detail that was lost at upscaling time
- Stop when we are back to the original size.
To obtain the detail lost during upscaling, we simply
take the original image, shrink it down, upscale it,
and compare the result to the (resized) original image.
"""


# Playing with these hyperparameters will also allow you to achieve new effects
step = 0.01  # Gradient ascent step size
num_octave = 4  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 30  # Number of ascent steps per scale
max_loss = 40.

img = preprocess_image(base_image_path)
if K.image_data_format() == 'channels_first':
    original_shape = img.shape[2:]
else:
    original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)

result_prefix = '/christmas_result'
print(result_prefix)
save_img(result_prefix + '.png', deprocess_image(np.copy(img)))
print(result_prefix,".png finised")

Model loaded.


<ipython-input-34-136b0c51834c>:17: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (layer_name in layer_dict.keys(),


Processing image shape (349, 233)
..Loss value at 0 : 0.00018197234
..Loss value at 1 : 0.00019621156
..Loss value at 2 : 0.00021155822
..Loss value at 3 : 0.00022794736
..Loss value at 4 : 0.00024581625
..Loss value at 5 : 0.00026533136
..Loss value at 6 : 0.0002868134
..Loss value at 7 : 0.00031029145
..Loss value at 8 : 0.00033600704
..Loss value at 9 : 0.00036423342
..Loss value at 10 : 0.00039497172
..Loss value at 11 : 0.0004284954
..Loss value at 12 : 0.00046498494
..Loss value at 13 : 0.00050528767
..Loss value at 14 : 0.00054982357
..Loss value at 15 : 0.0005992217
..Loss value at 16 : 0.000654005
..Loss value at 17 : 0.0007140435
..Loss value at 18 : 0.0007810135
..Loss value at 19 : 0.00085496245
..Loss value at 20 : 0.00093742507
..Loss value at 21 : 0.001029186
..Loss value at 22 : 0.0011313895
..Loss value at 23 : 0.0012458694
..Loss value at 24 : 0.0013733014
..Loss value at 25 : 0.0015156983
..Loss value at 26 : 0.0016733896
..Loss value at 27 : 0.0018497221
..Loss valu

C:\ProgramData\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


..Loss value at 0 : 0.00044504573
..Loss value at 1 : 0.0004567784
..Loss value at 2 : 0.00046880924
..Loss value at 3 : 0.00048113707
..Loss value at 4 : 0.00049375795
..Loss value at 5 : 0.0005067272
..Loss value at 6 : 0.00052009156
..Loss value at 7 : 0.0005338323
..Loss value at 8 : 0.0005479143
..Loss value at 9 : 0.00056236313
..Loss value at 10 : 0.0005772544
..Loss value at 11 : 0.0005925812
..Loss value at 12 : 0.0006083669
..Loss value at 13 : 0.00062456937
..Loss value at 14 : 0.00064113265
..Loss value at 15 : 0.0006580943
..Loss value at 16 : 0.0006754571
..Loss value at 17 : 0.000693318
..Loss value at 18 : 0.00071165647
..Loss value at 19 : 0.0007304296
..Loss value at 20 : 0.0007496801
..Loss value at 21 : 0.00076954276
..Loss value at 22 : 0.0007899672
..Loss value at 23 : 0.00081098056
..Loss value at 24 : 0.0008325358
..Loss value at 25 : 0.0008547279
..Loss value at 26 : 0.00087756297
..Loss value at 27 : 0.0009010673
..Loss value at 28 : 0.0009252604
..Loss value 

In [ ]:
print('Model loaded.')

# Get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model2.layers])

# Define the loss.
loss = K.variable(0.)
for layer_name in settings['features']:
    # Add the L2 norm of the features of a layer to the loss.
    assert (layer_name in layer_dict.keys(),
            'Layer ' + layer_name + ' not found in model.')
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(x), 'float32'))
    if K.image_data_format() == 'channels_first':
        loss += coeff * K.sum(K.square(x[:, :, 2: -2, 2: -2])) / scaling
    else:
        loss += coeff * K.sum(K.square(x[:, 2: -2, 2: -2, :])) / scaling

# Compute the gradients of the dream wrt the loss.
grads = K.gradients(loss, dream)[0]
# Normalize gradients.
grads /= K.maximum(K.mean(K.abs(grads)), K.epsilon())

# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)



"""Process:
- Load the original image.
- Define a number of processing scales (i.e. image shapes),
    from smallest to largest.
- Resize the original image to the smallest scale.
- For every scale, starting with the smallest (i.e. current one):
    - Run gradient ascent
    - Upscale image to the next scale
    - Reinject the detail that was lost at upscaling time
- Stop when we are back to the original size.
To obtain the detail lost during upscaling, we simply
take the original image, shrink it down, upscale it,
and compare the result to the (resized) original image.
"""


# Playing with these hyperparameters will also allow you to achieve new effects
step = 0.05  # Gradient ascent step size
num_octave = 4  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 30  # Number of ascent steps per scale
max_loss = 40.

img = preprocess_image(base_image_path)
if K.image_data_format() == 'channels_first':
    original_shape = img.shape[2:]
else:
    original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)

result_prefix = '/christmas_result'
print(result_prefix)
save_img(result_prefix + '.png', deprocess_image(np.copy(img)))
print(result_prefix,".png finised")

Model loaded.


<ipython-input-35-206b822e851b>:10: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (layer_name in layer_dict.keys(),


Processing image shape (349, 233)
..Loss value at 0 : 0.00018197234
..Loss value at 1 : 0.00019621156
..Loss value at 2 : 0.00021155822
..Loss value at 3 : 0.00022794736
..Loss value at 4 : 0.00024581625
..Loss value at 5 : 0.00026533136
..Loss value at 6 : 0.0002868134
..Loss value at 7 : 0.00031029145
..Loss value at 8 : 0.00033600704
..Loss value at 9 : 0.00036423342
..Loss value at 10 : 0.00039497172
..Loss value at 11 : 0.0004284954
..Loss value at 12 : 0.00046498494
..Loss value at 13 : 0.00050528767
..Loss value at 14 : 0.00054982357
..Loss value at 15 : 0.0005992217
..Loss value at 16 : 0.000654005
..Loss value at 17 : 0.0007140435
..Loss value at 18 : 0.0007810135
..Loss value at 19 : 0.00085496245
..Loss value at 20 : 0.00093742507
..Loss value at 21 : 0.001029186
..Loss value at 22 : 0.0011313895
..Loss value at 23 : 0.0012458694
..Loss value at 24 : 0.0013733014
..Loss value at 25 : 0.0015156983
..Loss value at 26 : 0.0016733896
..Loss value at 27 : 0.0018497221
..Loss valu

C:\ProgramData\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


..Loss value at 0 : 0.00044504573
..Loss value at 1 : 0.0004567784
..Loss value at 2 : 0.00046880924
..Loss value at 3 : 0.00048113707
..Loss value at 4 : 0.00049375795
..Loss value at 5 : 0.0005067272
..Loss value at 6 : 0.00052009156
..Loss value at 7 : 0.0005338323
..Loss value at 8 : 0.0005479143
..Loss value at 9 : 0.00056236313
..Loss value at 10 : 0.0005772544
..Loss value at 11 : 0.0005925812
..Loss value at 12 : 0.0006083669
..Loss value at 13 : 0.00062456937
..Loss value at 14 : 0.00064113265
..Loss value at 15 : 0.0006580943
..Loss value at 16 : 0.0006754571
..Loss value at 17 : 0.000693318
..Loss value at 18 : 0.00071165647
..Loss value at 19 : 0.0007304296
..Loss value at 20 : 0.0007496801
..Loss value at 21 : 0.00076954276
..Loss value at 22 : 0.0007899672
..Loss value at 23 : 0.00081098056
..Loss value at 24 : 0.0008325358
..Loss value at 25 : 0.0008547279
